In [2]:
import moto
import boto3

The formatting for tables is pretty verbose - better to have this in its own file

In [34]:
%%file dynamodb_table_format.json
{
  "TableName": "foobar",
  "AttributeDefinitions": [
    {
      "AttributeName": "Artist",
      "AttributeType": "S"
    },
    {
      "AttributeName": "SongTitle",
      "AttributeType": "S"
    }
  ],
  "KeySchema": [
    {
      "AttributeName": "Artist",
      "KeyType": "HASH"
    },
    {
      "AttributeName": "SongTitle",
      "KeyType": "RANGE"
    }
  ],
  "BillingMode": "PAY_PER_REQUEST",
  "Tags": [
    {
      "Key": "environment",
      "Value": "test"
    }
  ]
}

Overwriting dynamodb_table_format.json


Let's make a mock table and push some stuff into it

In [39]:
import json

with open('dynamodb_table_format.json', 'r') as src:
    table_fmt = json.load(src)

with moto.mock_dynamodb2():
    dynamodb = boto3.resource("dynamodb")
    table = dynamodb.create_table(**table_fmt)
    print(table)
    print(list(dynamodb.tables.all()))

dynamodb.Table(name='foobar')
[dynamodb.Table(name='foobar')]
